#   Tool Calling (Function Calling) + 에이전트(Agent) 개념

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [19]:
import os
from glob import glob

from pprint import pprint
import json

---

## **Tool Calling**

- **Tool Calling**은 LLM이 외부 시스템과 상호작용하기 위한 **함수 호출 메커니즘**

- LLM은 정의된 도구나 함수를 통해 **외부 시스템과 통신**하고 작업을 수행

- **Tool calling**은 모델이 시스템과 직접 상호작용할 수 있게 하는 기능

- **구조화된 출력**을 통해 API나 데이터베이스와 같은 시스템 요구사항 충족

- **스키마 기반 응답**으로 시스템 간 효율적 통신 가능


![Tool Calling Concept](https://python.langchain.com/assets/images/tool_calling_concept-552a73031228ff9144c7d59f26dedbbf.png)


[참조] https://python.langchain.com/docs/concepts/tool_calling/

---

### 1. **Tool Creation** (`@tool` 데코레이터 사용)

- **@tool 데코레이터**로 함수에 스키마 정보 추가

- **함수와 스키마** 간 자동 연결로 도구 생성

In [20]:
from langchain_core.tools import tool
from typing import Literal

@tool
def get_weather(city: Literal["서울", "부산", "대구", "인천", "광주"]):
    """한국 주요 도시의 날씨 정보를 가져옵니다."""
    weather_data = {
        "서울": "맑음",
        "부산": "흐림",
        "대구": "맑음",
        "인천": "비",
        "광주": "구름많음"
    }
    
    if city in weather_data:
        return f"{city}은(는) {weather_data[city]}"
    else:
        raise AssertionError("지원하지 않는 도시입니다")

---
### **[실습]**

- **크로마 DB 재사용**을 위해 **프로젝트 2**의 `chromadb` 디렉토리를 현재 프로젝트 폴더에 **복사하여 활용**

- 이전 프로젝트의 **테슬라, 리비안 데이터**를 동일한 **임베딩 모델**로 검색 가능하도록 구성

- 사용자 정의 **문서 검색 도구**를 구현 (`@tool` 데코레이터 사용)

In [21]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

print(f"ChromaDB에 저장된 문서 개수: {chroma_db._collection.count()}")



# 검색기 지정하여 테스트 
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")




# DB 검색하는 사용자 정의 도구 생성
from langchain.tools import tool

@tool
def search_db(query: str):
    """리비안, 테슬라 회사에 대한 정보를 관련 데이터베이스에서 검색합니다."""
    return chroma_k_retriever.invoke(query)

# 도구 실행
search_db.invoke("리비안은 언제 사업을 시작했나요?")





"""
# 검색기 지정하여 테스트 
retriever = chroma_db.as_retriever(search_kwargs={"k": 3})

# 테스트 쿼리
test_docs = retriever.invoke("테슬라의 전기차 기술은 어떤가요?")
print(f"\n검색된 문서 개수: {len(test_docs)}")
print(f"첫 번째 문서 미리보기:\n{test_docs[0].page_content[:200]}...")

# DB 검색하는 사용자 정의 도구 생성
@tool
def search_documents(query: str) -> str:
    '''ChromaDB 벡터 저장소에서 테슬라와 리비안 관련 문서를 검색합니다.
    
    Args:
        query: 검색할 질문이나 키워드
        
    Returns:
        검색된 문서 내용들을 결합한 문자열
    '''
    retriever = chroma_db.as_retriever(search_kwargs={"k": 3})
    docs = retriever.invoke(query)
    
    # 검색된 문서들을 하나의 문자열로 결합
    result = "\n\n---\n\n".join([doc.page_content for doc in docs])
    return result

# 도구 테스트
print("\n=== 문서 검색 도구 테스트 ===")
result = search_documents.invoke("리비안의 주요 특징은?")
print(result[:300] + "...")
"""

ChromaDB에 저장된 문서 개수: 39
쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- [출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러 [출처: data/리비안_KR.md]
- [출처] 이 문서는 리비안에 대한 문서입니다.
----------------------------------
Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.

**주요 정보:** [출처: data/리비안_KR.md]


'\n# 검색기 지정하여 테스트 \nretriever = chroma_db.as_retriever(search_kwargs={"k": 3})\n\n# 테스트 쿼리\ntest_docs = retriever.invoke("테슬라의 전기차 기술은 어떤가요?")\nprint(f"\n검색된 문서 개수: {len(test_docs)}")\nprint(f"첫 번째 문서 미리보기:\n{test_docs[0].page_content[:200]}...")\n\n# DB 검색하는 사용자 정의 도구 생성\n@tool\ndef search_documents(query: str) -> str:\n    \'\'\'ChromaDB 벡터 저장소에서 테슬라와 리비안 관련 문서를 검색합니다.\n\n    Args:\n        query: 검색할 질문이나 키워드\n\n    Returns:\n        검색된 문서 내용들을 결합한 문자열\n    \'\'\'\n    retriever = chroma_db.as_retriever(search_kwargs={"k": 3})\n    docs = retriever.invoke(query)\n\n    # 검색된 문서들을 하나의 문자열로 결합\n    result = "\n\n---\n\n".join([doc.page_content for doc in docs])\n    return result\n\n# 도구 테스트\nprint("\n=== 문서 검색 도구 테스트 ===")\nresult = search_documents.invoke("리비안의 주요 특징은?")\nprint(result[:300] + "...")\n'

---

### 2. **Tool Binding** (모델에 Tool 연결)

- **모델-도구 연결**로 입력 스키마 자동 인식

- **스키마 기반 검증**으로 올바른 입력 보장

In [22]:
from langchain.chat_models import init_chat_model
model = init_chat_model(
    "openai:gpt-4.1-nano",
    # Kwargs passed to the model:
    temperature=0.7,
    timeout=30,
    max_tokens=1000,
)

# 도구 목록
tools = [get_weather, search_db]

# 도구를 모델에 바인딩 (bind_tools 메소드 사용)
model_with_tools = model.bind_tools(tools)

"""
from langchain_openai import ChatOpenAI

# 모델
model = ChatOpenAI(model="gpt-4.1-nano",temperature=0)

# 도구 목록
tools = [get_weather]

# 도구를 모델에 바인딩 (bind_tools 메소드 사용)
model_with_tools = model.bind_tools([get_weather])
"""

'\nfrom langchain_openai import ChatOpenAI\n\n# 모델\nmodel = ChatOpenAI(model="gpt-4.1-nano",temperature=0)\n\n# 도구 목록\ntools = [get_weather]\n\n# 도구를 모델에 바인딩 (bind_tools 메소드 사용)\nmodel_with_tools = model.bind_tools([get_weather])\n'

In [23]:
# 사용자 쿼리를 모델에 전달
result = model_with_tools.invoke("서울 날씨 어때?")

print(result)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 109, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1f35c1788c', 'id': 'chatcmpl-CT6RicEbJFRgNtBSJGf7fns3LEQNb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--a9fbcd6a-e048-43e8-a173-8bfd5004c816-0' tool_calls=[{'name': 'get_weather', 'args': {'city': '서울'}, 'id': 'call_DvSK2oX3eat9PVOr9JSL8obk', 'type': 'tool_call'}] usage_metadata={'input_tokens': 109, 'output_tokens': 14, 'total_tokens': 123, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


---
### **[실습]**

- 앞의 실습에서 정의한 도구를 llm 모델에 **바인딩** 처리
- 사용자 쿼리를 입력하여 출력 결과를 search_result 변수에 저장하고 출력 확인

In [24]:
# 도구를 모델에 바인딩
model_with_search = model.bind_tools([search_db])

# 도구를 사용하여 쿼리 실행
search_result = model_with_search.invoke("테슬라의 배터리 기술에 대해 알려주세요")

# 결과 출력
print("=== 모델 응답 결과 ===")
print(search_result)
print("\n" + "="*100)
print("\n=== Tool Calls 확인 ===")
pprint(search_result.tool_calls)

=== 모델 응답 결과 ===
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 68, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_4c0b74f64c', 'id': 'chatcmpl-CT6RjQFv4m4CeL9EL7alWsjBq2sRf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--deee2d46-4e73-442c-84dc-fbc32a271fc5-0' tool_calls=[{'name': 'search_db', 'args': {'query': '테슬라 배터리 기술'}, 'id': 'call_WxsDdwoMup19o9i1H5pEoJSF', 'type': 'tool_call'}] usage_metadata={'input_tokens': 68, 'output_tokens': 20, 'total_tokens': 88, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


=== Tool Calls 확인 ===
[{'args'

---

### 3. **Tool Calling** (모델이 Tool을 사용하는 경우)

- **스키마 기반 응답** 생성으로 정확한 입력 형식 준수

- **자동 유효성 검증**으로 오류 방지

- **구조화된 출력** 생성으로 시스템 호환성 보장

In [25]:
# 결과 출력
for k in dict(result).keys():
    print(f"{k}: ")
    print(dict(result)[k])
    print("-"*100)

content: 

----------------------------------------------------------------------------------------------------
additional_kwargs: 
{'refusal': None}
----------------------------------------------------------------------------------------------------
response_metadata: 
{'token_usage': {'completion_tokens': 14, 'prompt_tokens': 109, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1f35c1788c', 'id': 'chatcmpl-CT6RicEbJFRgNtBSJGf7fns3LEQNb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}
----------------------------------------------------------------------------------------------------
type: 
ai
-----------------------------------------------------------------------------------------------

In [26]:
# tool_calls 출력
pprint(result.tool_calls)

[{'args': {'city': '서울'},
  'id': 'call_DvSK2oX3eat9PVOr9JSL8obk',
  'name': 'get_weather',
  'type': 'tool_call'}]


---
### **[실습]**

- search_result 변수에 저장된 tool call 내역을 확인

In [27]:
# 결과 출력
print("=== search_result의 모든 속성 ===")
for k in dict(search_result).keys():
    print(f"\n{k}: ")
    print(dict(search_result)[k])
    print("-"*100)

# tool_calls 출력
print("\n=== Tool Calls 상세 정보 ===")
pprint(search_result.tool_calls)

=== search_result의 모든 속성 ===

content: 

----------------------------------------------------------------------------------------------------

additional_kwargs: 
{'refusal': None}
----------------------------------------------------------------------------------------------------

response_metadata: 
{'token_usage': {'completion_tokens': 20, 'prompt_tokens': 68, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_4c0b74f64c', 'id': 'chatcmpl-CT6RjQFv4m4CeL9EL7alWsjBq2sRf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}
----------------------------------------------------------------------------------------------------

type: 
ai
----------------------------------------------------------------

---

### 4. **Tool Execution**  (Tool이 호출된 경우 실행)

- **인자 기반 실행**으로 도구 기능 수행

- **모델 제공 파라미터**로 자동화된 실행

- **실행 결과** 처리 및 반환

In [28]:
# 함수의 인자를 직접 전달하는 방식으로 실행 -> 도구를 직접 호출
get_weather.invoke("서울")

'서울은(는) 맑음'

In [29]:
# ToolCall 객체를 전달 전달하는 방식으로 실행 -> ToolMessage 객체를 반환
get_weather.invoke(result.tool_calls[0])

ToolMessage(content='서울은(는) 맑음', name='get_weather', tool_call_id='call_DvSK2oX3eat9PVOr9JSL8obk')

---

###  Tool Calling 사용 시 **고려사항**

- **모델 호환성**이 Tool Calling 성능에 직접 영향

- **명확한 도구 정의**가 모델의 이해도와 활용도 향상

- **단순한 기능**의 도구가 더 효과적으로 작동

- **과다한 도구**는 모델 성능 저하 유발

---
### **[실습]**

- search_result 변수에 저장된 tool call 내역을 직접 도구에 적용하여 실행

In [30]:
# ToolCall 객체를 전달하는 방식으로 실행 -> ToolMessage 객체를 반환
if search_result.tool_calls:
    # 첫 번째 tool call을 사용하여 도구 실행
    tool_message = search_db.invoke(search_result.tool_calls[0])
    
    print("=== ToolMessage 객체 ===")
    print(f"타입: {type(tool_message)}")
    print(f"\n도구 이름: {tool_message.name}")
    print(f"Tool Call ID: {tool_message.tool_call_id}")
    print(f"\n검색 결과 내용 (처음 500자):\n{str(tool_message.content)[:500]}...")
else:
    print("도구 호출이 없습니다.")

=== ToolMessage 객체 ===
타입: <class 'langchain_core.messages.tool.ToolMessage'>

도구 이름: search_db
Tool Call ID: call_WxsDdwoMup19o9i1H5pEoJSF

검색 결과 내용 (처음 500자):
[Document(id='5f7b0a65-c409-47ab-84cb-d8b58229400c', metadata={'company': '테슬라', 'language': 'ko', 'source': 'data/테슬라_KR.md'}, page_content='[출처] 이 문서는 테슬라에 대한 문서입니다.\n----------------------------------\n## 비즈니스 전략\n\nTesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하는 것입니다. Tesla는 자동차의 하드웨어를 지속적으로 업데이트하고 웹사이트와 회사 소유 매장을 통해 직접 차량을 판매합니다. Tesla는 수직적으로 통합되어 많은 구성 요소를 자체 개발합니다. Tesla는 일반적으로 지속 가능한 에너지 채택을 촉진하기 위해 경쟁 업체가 자사 기술을 라이선스하도록 허용합니다.\n\n## 기술\n\n### 배터리\n\nTesla는 CATL, LG E...


---

## **Agent**

- **LLM(대규모 언어 모델)** 을 의사결정 엔진으로 사용하여 작업을 수행하는 시스템

- 모델은 입력된 데이터를 분석하여 **맥락에 맞는 의사결정**을 수행

- 시스템은 사용자의 요청을 이해하고 **적절한 해결책**을 제시

- 복잡한 작업을 자동화하여 **업무 효율성**을 높일 수 있음 

---

### **create_agent** 

- **create_agent**는 LangChain v1.0의 표준 에이전트 생성 함수

- LangGraph를 기반으로 구축되어 **영속성, 스트리밍, Human-in-the-loop** 등의 기능을 자동 지원

- **미들웨어**를 통한 유연한 커스터마이징 가능

`(1) 추가 도구 정의`

- **@tool 데코레이터**를 사용해 계산(파이썬 코드 실행) 기능을 가진 **커스텀 도구를 정의**

- 데코레이터를 통해 함수가 **Tool Calling 시스템에 등록**되어 LLM이 호출 가능

In [31]:
@tool
def calculate(expression: str) -> float:
    """수학 계산을 수행합니다."""
    return eval(expression)

In [32]:
# 도구 실행 
calculate.invoke("3+2")

5

`(2) create_agent로 에이전트 생성`

- **create_agent**는 모델, 도구, 시스템 프롬프트를 받아 에이전트를 생성

- **system_prompt** 매개변수로 에이전트의 역할과 행동 방식을 정의

- LangGraph 기반으로 구축되어 자동으로 메시지 기록, 도구 실행 루프 등을 관리

In [33]:
# LangGraph의 create_react_agent 사용
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 도구 목록
tools = [get_weather, calculate]

# 시스템 프롬프트를 포함한 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 사용자의 요청을 처리하는 AI Assistant입니다."),
    ("placeholder", "{messages}"),
])

# 에이전트 생성 (prompt 파라미터 사용)
agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt=prompt
)

print("에이전트가 성공적으로 생성되었습니다.")

에이전트가 성공적으로 생성되었습니다.


C:\Users\kaydash\AppData\Local\Temp\ipykernel_41264\2589284234.py:19: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


`(3) 에이전트 실행`

- **invoke** 메서드로 에이전트를 실행

- **messages** 키에 대화 메시지 리스트를 전달

- 에이전트는 자동으로 필요한 도구를 호출하고 최종 응답을 생성

In [34]:
# 에이전트 실행
response = agent.invoke(
    {"messages": [{"role": "user", "content": "서울의 날씨는 어떤가요?"}]},
)

# 에이전트 실행 결과 출력
pprint(response)

{'messages': [HumanMessage(content='서울의 날씨는 어떤가요?', additional_kwargs={}, response_metadata={}, id='c6008f62-3cbb-49a6-b903-8040f153eeac'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 112, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CT6RkA5KXoqcW6hXhO5Rt9jKrizXf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--aefc7d63-c4f9-4b71-a1a9-5a28ec85b2f9-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '서울'}, 'id': 'call_Kb0PQKayN2FCfdI9wpnrkY3a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 112, 'output_tokens': 14, 'total_tokens

In [35]:
for msg in response['messages']:
    msg.pretty_print()

================================ Human Message =================================

서울의 날씨는 어떤가요?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_Kb0PQKayN2FCfdI9wpnrkY3a)
 Call ID: call_Kb0PQKayN2FCfdI9wpnrkY3a
  Args:
    city: 서울
================================= Tool Message =================================
Name: get_weather

서울은(는) 맑음
================================== Ai Message ==================================

서울의 날씨는 맑습니다.


In [36]:
# 계산 도구를 사용하는 예제
response = agent.invoke(
    {"messages": [{"role": "user", "content": "32 더하기 18은 얼마인가요?"}]},
)

# 에이전트 실행 결과 출력
for msg in response['messages']:
    msg.pretty_print()

================================ Human Message =================================

32 더하기 18은 얼마인가요?
================================== Ai Message ==================================
Tool Calls:
  calculate (call_kkKFJpKXjbfWcrl2NE1qUsjo)
 Call ID: call_kkKFJpKXjbfWcrl2NE1qUsjo
  Args:
    expression: 32 + 18
================================= Tool Message =================================
Name: calculate

50
================================== Ai Message ==================================

32 더하기 18은 50입니다.


`(4) 중간 단계 확인하기`

- **stream_mode="values"** 를 사용하여 에이전트의 실행 과정을 스트리밍으로 확인

- 각 단계에서 모델의 사고 과정과 도구 호출 내역을 추적 가능

In [37]:
# 스트리밍 모드로 중간 단계 확인
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "32 곱하기 18은 얼마인가요?"}]},
    stream_mode="values"
):
    # 각 단계의 메시지 출력
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

32 곱하기 18은 얼마인가요?
================================== Ai Message ==================================
Tool Calls:
  calculate (call_jK9jOWtyi0GefLc5TRicXwvx)
 Call ID: call_jK9jOWtyi0GefLc5TRicXwvx
  Args:
    expression: 32 * 18
================================= Tool Message =================================
Name: calculate

576
================================== Ai Message ==================================

32 곱하기 18은 576입니다.


---
### **[실습]**

- 이전 [실습]에서 구현한 **문서 검색 도구**를 사용하여 에이전트 구현 및 실행 

In [38]:
# 필요한 라이브러리 import
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate

# 사용자 정의 시스템 프롬프트
system_message = """당신은 테슬라와 리비안 전기차에 대한 전문 상담 AI입니다.
사용자의 질문에 대해 ChromaDB에 저장된 문서를 검색하여 정확한 정보를 제공하세요.

검색된 문서 내용을 바탕으로:
1. 질문에 대한 직접적인 답변을 제공하세요
2. 관련된 세부 정보를 포함하세요
3. 검색 결과가 없으면 솔직하게 알려주세요
"""

# 프롬프트 템플릿 생성
doc_prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("placeholder", "{messages}"),
])

# 에이전트 생성 (prompt 파라미터 사용)
doc_agent = create_react_agent(
    model=llm,
    tools=[search_db],
    prompt=doc_prompt
)

print("문서 검색 에이전트가 생성되었습니다.\n")

# 에이전트 실행
print("=== 에이전트 실행 예제 1: 테슬라 관련 질문 ===")
response1 = doc_agent.invoke(
    {"messages": [{"role": "user", "content": "테슬라의 주요 기술적 특징은 무엇인가요?"}]}
)

# 결과 출력
for msg in response1['messages']:
    msg.pretty_print()

print("\n" + "="*100)
print("\n=== 에이전트 실행 예제 2: 리비안 관련 질문 ===")
response2 = doc_agent.invoke(
    {"messages": [{"role": "user", "content": "리비안 전기차의 특징을 알려주세요"}]}
)

# 최종 답변만 출력
final_message = response2['messages'][-1]
print(f"\n최종 답변:\n{final_message.content}")

문서 검색 에이전트가 생성되었습니다.

=== 에이전트 실행 예제 1: 테슬라 관련 질문 ===


C:\Users\kaydash\AppData\Local\Temp\ipykernel_41264\453513859.py:22: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  doc_agent = create_react_agent(


================================ Human Message =================================

테슬라의 주요 기술적 특징은 무엇인가요?
================================== Ai Message ==================================
Tool Calls:
  search_db (call_zWiXdhcSQhqQjDi9SkYZZ4nH)
 Call ID: call_zWiXdhcSQhqQjDi9SkYZZ4nH
  Args:
    query: 테슬라 주요 기술적 특징
================================= Tool Message =================================
Name: search_db

[Document(id='03c822e1-b421-4811-b362-e68fec329fdf', metadata={'company': '테슬라', 'language': 'ko', 'source': 'data/테슬라_KR.md'}, page_content='[출처] 이 문서는 테슬라에 대한 문서입니다.\n----------------------------------\n### 소프트웨어\n\nTesla는 무선 업데이트를 사용하여 강력한 온보드 컴퓨터를 통해 기능을 추가하거나 문제를 해결합니다.\n\n### 모터\n\nTesla는 유도 모터와 동기 릴럭턴스 모터(SynRM) 특성을 가진 내부 영구 자석(IPM) 모터를 만듭니다.\n\n### 북미 충전 표준\n\n북미 충전 표준(NACS)은 Tesla에서 개발한 전기 자동차 충전 커넥터 시스템입니다.\n\n### Autopilot 및 완전 자율 주행\n\nTesla Autopilot은 Tesla에서 개발한 고급 운전자 지원 시스템(ADAS)으로, 부분적인 차량 자동화를 의미합니다.\n\n### 로봇 공학'), Document(id='b2eb6d62-9519-4875-8f22-ce6b4c